In [5]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r gdal_req.txt

# Restart the runtime by ending the process.
exit()

In [6]:
import numpy as np
import tensorflow as tf
from google.cloud import storage
import google.auth
from serving.constants import  BUCKET, IMG_SOURCE_PREFIX, HIST_DEST_PREFIX, NUM_BANDS, HIST_BINS_LIST, SCALE, CROP
from serving.common import list_blobs_with_prefix
import logging
import io

In [8]:
# SETUP
credentials, _ = google.auth.default()
bucket_name = BUCKET
directory_prefix = IMG_SOURCE_PREFIX
output_prefix = HIST_DEST_PREFIX

client = storage.Client()
bucket = client.get_bucket(bucket_name)

logging.basicConfig(filename="cerate_dataset.log",level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

counties = ["Lancaster", "Crawford", "Orleans", "Tulare", "Wharton", "Story", "Canyon", "Kit Carson"]
years = range(2017,2019)
months = [5,7,9]

In [40]:
year = 2017
county = "Tulare"
hist_name_base = f"{HIST_DEST_PREFIX}{county}/{year}"

arrays = {}
for month in months:
    file_name = f"{hist_name_base}/{month}-{month+1}/{SCALE}/hist.npy"
    hist_blob = bucket.blob(file_name)    
    print(hist_blob)
    if hist_blob.name.endswith('.npy'):
            # Download the content of the blob
            content = hist_blob.download_as_bytes()

            # Use BytesIO to create a file-like object in memory
            binary_data = io.BytesIO(content)

            # Load the NumPy array from the binary data
            array = np.load(binary_data)
            tensor = tf.convert_to_tensor(array)

            # Store in our dictionary
            arrays[hist_blob.name] = tensor

<Blob: vgnn, histograms/Tulare/2017/5-6/100/hist.npy, None>
<Blob: vgnn, histograms/Tulare/2017/7-8/100/hist.npy, None>
<Blob: vgnn, histograms/Tulare/2017/9-10/100/hist.npy, None>


In [34]:
arrays["histograms/Tulare/2017/5-6/100/hist.npy"].shape

(416,)

In [47]:
list(arrays.values())

[<tf.Tensor: shape=(416,), dtype=int64, numpy=
 array([   0,    0,    0,    2, 2369, 1048,  380,  270,  150,   76,   52,
           7,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 3095,  632,  329,  193,   89,   15,    3,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         545, 2352,  464,  292,  169,  173,  142,   78,   41,   37,   50,
           7,    5,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  463, 3139,  452,  260,   39,    1,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    8,    5,    7,
           3,    3,    4,    3,   15,   62,   36,   27,   37,  10

In [53]:
big_tensor = tf.concat(list(arrays.values()), axis=0)

In [59]:
final_tensor = tf.reshape(big_tensor, [1,3,-1])

In [62]:
final_tensor[0,0,:]

<tf.Tensor: shape=(416,), dtype=int64, numpy=
array([   0,    0,    0,    2, 2369, 1048,  380,  270,  150,   76,   52,
          7,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3095,  632,  329,  193,   89,   15,    3,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        545, 2352,  464,  292,  169,  173,  142,   78,   41,   37,   50,
          7,    5,    1,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,  463, 3139,  452,  260,   39,    1,    2,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    8,    5,    7,
          3,    3,    4,    3,   15,   62,   36,   27,   37,  102,  179,
     